In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model, Sequential, losses, optimizers, metrics

from helper import plot_df, resample
from helper import decode_image, load_and_preprocess_image, load_tfrecord_dataset, parse_example
from helper import load_compressed_model

ModuleNotFoundError: No module named 'matplotlib'

In this coding exercise you will build a model that predicts 3 properties of an individual from a picture of their face: age, ethnicity, and gender.

This Dataframe gives us an idea of what we are working with.

In [ ]:
# Read the dataframe
df = pd.read_csv('data/face_data.csv')
df.head()

: 

In [12]:
# map integer encodings to class labels
# label ordering from the Kaggle dataset release:
names_ethnicity = ['white', 'black', 'asian', 'indian', 'other']
name_genders = ['male', 'female']

In [ ]:
plot_df(df)

: 

We'll use a helper function to `resample` to make sure the ethnicities are equally represented in a new Dataframe.\
Please don't change `OBSERVATION_PER_CLASS` from the default of 750.

In [ ]:
OBSERVATIONS_PER_CLASS = 750
df_resampled = resample(df, unbalanced_col='ethnicity', n_per_class=OBSERVATIONS_PER_CLASS)

: 

In [ ]:
plot_df(df_resampled)

: 

That looks a little better, though not perfect. We'll settle for that for now.

**Train / Validation / Test Split**

Using your resampled data and `train_test_split`, create a test set consisting of 20% of the original dataset. Then set aside 10% of what was left over from the first split as a validation set. Whatever remains will be your train set.

You should stratify your splits on both 'ethnicity' and 'gender' and use a random state of 109.

Store the results in `train_df`, `val_df`, and `test_df`.

In [ ]:
### TEST FUNCTION: test_split
# DO NOT REMOVE THE LINE ABOVE
# Split the dataframe into train and validation sets
# Create test set of 20% then set aside 10% of the training set for validation
# Use random state of 109
# Stratify on enthnicity and gender columns

# split the data into train+validation and test sets
train_val_df, test_df = train_test_split(resampled_df, test_size=0.2, random_state=109, stratify=resampled_df[['ethnicity', 'gender']])

# split the train+validation set into train and validation sets
train_df, val_df = train_test_split(train_val_df, test_size=0.1, random_state=109, stratify=train_val_df[['ethnicity', 'gender']])

: 

In [17]:
### TEST FUNCTION: test_stratify
# DO NOT REMOVE THE LINE ABOVE

**Creating Datasets**

For the sake of consistency, to minimize mememory and storage space, and to make trainign as fast as possible, we will provide pre-made train, validation, and test datasets in the form of TF Dataset pipelines.

These have been balanced for both ethnicity and gender within each ethnicity, similar to what you attempted above in the splitting.

Run the code below to create train, validation, and test datasets. 

Later you may want to play with the batch size, but remember to stay conservative to prevent the kernel from dying.

In [18]:
BATCH_SIZE = 32 # num images in a generator batch
IMG_DIM = (48, 48) # orignal images are 48x48 (model will require at least 32x32)
train_tfrecord_file = 'data/faces_train_balanced.tfrecord'
val_tfrecord_file = 'data/faces_val_balanced.tfrecord'
test_tfrecord_file = 'data/faces_test_balanced.tfrecord'

train_dataset = load_tfrecord_dataset(train_tfrecord_file, IMG_DIM, BATCH_SIZE)
val_dataset = load_tfrecord_dataset(val_tfrecord_file, IMG_DIM, BATCH_SIZE)
test_dataset = load_tfrecord_dataset(test_tfrecord_file, IMG_DIM, BATCH_SIZE)

We can see what the datasets produce with this code:

In [19]:
# inspect what the datasets produce
train_dataset.element_spec

The **inputs** are 48x48 grayscale images of faces 
(the single channel has been duplicated twice to make them 48x48x3 to work well with models trained on color images.)

The **target** variables match the order of the columns in the Dataframe above.
- The first target is the **age** variable (continous).
- The second target is **ethnicity** (categorical)
- The third target is **gender** (binary in this dataset)

**Transfer Learning with BabyNet**

The base of your model will be a very tiny CNN classifier pre-trained on the CFAR-10 dataset (~75% test accuracy). The model is 10% the size of MobileNetV3Small; We call it `BabyNet`. 

Load this pre-trained model from the `BabyNet.zip` file using the provided `load_compressed_model()` helper function, saving the model as `babynet`, and inspecting the summary.

In [20]:
### TEST FUNCTION: test_babynet_load
# DO NOT REMOVE THE LINE ABOVE
# Split the dataframe into train and validation sets
# Load the 'BabyNet' model
babynet = load_compressed_model(...)
babynet.summary()

**Chop Off the Top!**

We want to leverage the feature extraction ability of the convolutional section of the network (conv2d & pooling) since we think these features will generalize to our own task of predicting age, ethnicity, and gender from pictures of faces, (even though the CFAR-10 dataset contains no human faces!)

But we *do not care* about the original 'top' of BabyNet since these final layers are more related to the specific CFAR-10 classification task.

Create a new model, `base_model`, with the same `inputs` as `babynet` but whose outputs come from the final layer in the convolutional section of `babynet`. Be sure to also give your new model the name attribute 'base_model' when you construct it.

Printing the summary is always helpful!

In [21]:
### TEST FUNCTION: test_choptop
# DO NOT REMOVE THE LINE ABOVE
# your code here
base_model = ...

base_model.summary()

**Freeze Weights**

In our initial round of training we don *not* want to mangle the pre-trained weights because we think they will be useful. So you should **set the base model's weight to be not trainable**.


In [22]:
### TEST FUNCTION: test_freeze
# DO NOT REMOVE THE LINE ABOVE
# your code here


**Data Augmentation**

Create a small model to perform data augmentation. It should have one layer to perform a random, horizontal 'flip' followed by another layer to add a small amount of GaussianNoise. Store this model in the variable `data_augmentation` and give it the name attribute "data_aug".

**Hint:**
- See what layers are available in `tf.keras.layers`
- It maybe be easier for you if you also use an input "layer"
- Images are already normalized to be between 0 and 1 (what BabyNet expects) so the amount of noise you want to add should be very small.

In [23]:
### TEST FUNCTION: test_data_aug
# DO NOT REMOVE THE LINE ABOVE
# your code here
data_augmentation = 

In [24]:
# display data augmentation model summary
data_augmentation.summary()

Run the code below to visualize some augmentation examples.

In [25]:
# generate variants of the first image from the training dataset
for images, labels in train_dataset.take(1):
    plt.figure(figsize=(4, 4))
    first_image = images[0]
    for i in range(4):
        ax = plt.subplot(2, 2, i + 1)
        # apply the transformation layers
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0])
        plt.axis("off")
plt.tight_layout()

**Building the Model**

Construct your model. It's structure should look like this:
- input "layer"
- data augmentation
- base_model
- new intermediate layer(s)
- 3 output layers, give them the name attributes 'age_output', 'ethnicity_output', 'gender_output'

**Hint:**
- The output of the base model is 3D (64 3x3 feature maps). Your new intermediate layer(s) will need too convert this output to 1D before it can be passed on to your output layers. We saw *at least 2 different ways* of doing this in class. Again, see what `tf.keras.layers` has to offer.
- You can cause the kernel to crash (or just make your training very slow) if your model is too big. Start out small. You can pass all the tests in this notebook with less than 100k total parameters in your full network.


In [26]:
### TEST FUNCTION: test_output_count
# DO NOT REMOVE THE LINE ABOVE
# your code here
model = ...


In [27]:
### TEST FUNCTION: test_categorical_output
# DO NOT REMOVE THE LINE ABOVE

In [28]:
### TEST FUNCTION: test_age_output
# DO NOT REMOVE THE LINE ABOVE

In [29]:
### TEST FUNCTION: test_ethnicity_output
# DO NOT REMOVE THE LINE ABOVE

In [30]:
### TEST FUNCTION: test_gender_output
# DO NOT REMOVE THE LINE ABOVE

In [31]:
### TEST FUNCTION: test_data_aug_insert
# DO NOT REMOVE THE LINE ABOVE

In [32]:
### TEST FUNCTION: test_base_model_insert
# DO NOT REMOVE THE LINE ABOVE

In [33]:
# display summary
model.summary()

**Compile the Model**

When dealing with multiple outputs it is useful to pass dictionaries for both the `loss` and `metrics` arguments of `compile`. The dictionary keys are the layer names and their values are the losses or metrics themselves. Choose appropriate losses for each of the 3 outputs. You should monitor the following metrics:
- age: mean absolute error
- ethnicity: accuracy
- gender: accuracy

Create the dictionaries `loss` and `metrics` to pass to the `compile function`.

**Note:** The values in the dictionaries should be *strings* representing the relevant loss or metric (Keras understands these). These strings are expected to be lowercase and any spaces should be replaced by underscores. You can also use abbreviations for some options which should still appease the test cases.

For the **optimizer**, we recomment the Adam optimizer, though you might want to try something slighlty higher than the default learning rate.

Save your optimizer as `optimizer1`.

In [35]:
### TEST FUNCTION: test_optimizer1
# DO NOT REMOVE THE LINE ABOVE
# your code here
optimizer1 = ...
loss = ...
metrics = ...

In [ ]:
### TEST FUNCTION: test_model_loss
# DO NOT REMOVE THE LINE ABOVE

In [ ]:
### TEST FUNCTION: test_model_metrics
# DO NOT REMOVE THE LINE ABOVE

**Initial Training**

Train the new layers you added to the base model. You'll want to do at least 3 epochs. More would likely help, but start small while you're still testing things.

In [27]:
### TEST FUNCTION: test_initial_training
# DO NOT REMOVE THE LINE ABOVE
# your code here
initial_epochs = ...

history1 = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=val_dataset)

**Unfreeze Base Model & Recompile with New Optimizer**

Now we will fine-tune the base model. First you'll need to unfreeze its weights. Remember that you need to recompile the model for the change to take effect. When you print the summary the model should now show many more trainable parameters in the summary.

When recompiling in preparation for fine tuning you should choose a sensible **learning rate** for the new optimizer. Call this new optimizer, `optimizer2`.

In [29]:
### TEST FUNCTION: test_unfreeze
# DO NOT REMOVE THE LINE ABOVE
# unfreeze the base model
# your code here
optimizer2 = ...


In [ ]:
### TEST FUNCTION: test_optimizer2
# DO NOT REMOVE THE LINE ABOVE

In [31]:
# display summary
model.summary()

**Fine-Tuning**

Now that *all* the weights in the network are trainable, train your model for a few more epochs. You'll probably want to do at least 2, but more may be beneficial depending on your initial training and model architecture.

**Hint:** remember that adjusting the weights of the pre-trained model also has the potential to be *destructive* if the learning rate is too high.

In [32]:
### TEST FUNCTION: test_fine_tuning
# DO NOT REMOVE THE LINE ABOVE
# your code here
fine_tune_epochs = ...
total_epochs = initial_epochs + fine_tune_epochs

history2 = model.fit(train_dataset,
                    initial_epoch=initial_epochs,  # Resume training
                    epochs=total_epochs,
                    validation_data=val_dataset)

**Display Full Training History**

In [33]:
full_history = {}
for key in history1.history.keys():
    full_history[key] = history1.history[key] + history2.history[key]

fig, axs = plt.subplots(1,3, figsize=(9,4))
axs = axs.flatten()

axs[0].plot(full_history['val_age_output_'+metrics['age_output']], label='train')
axs[0].plot(full_history['age_output_'+metrics['age_output']], label='validation')
axs[0].set_ylabel('MAE')
axs[0].set_title('Age')

axs[1].plot(full_history['ethnicity_output_'+metrics['ethnicity_output']], label='train')
axs[1].plot(full_history['val_ethnicity_output_'+metrics['ethnicity_output']], label='validation')
axs[1].axhline(0.2, c='r', ls=':', label='random chance')
axs[1].set_ylabel('Accuracy')
axs[1].set_title('Ethnicity')

axs[2].plot(full_history['gender_output_'+metrics['gender_output']], label='train')
axs[2].plot(full_history['val_gender_output_'+metrics['gender_output']], label='validation')
axs[2].axhline(0.5, c='r', ls=':', label='random chance')
axs[2].set_ylabel('Accuracy')
axs[2].set_title('Gender')

for ax in axs:
    ax.set_xlabel("Epoch")
    ax.set_xticks(list(range(total_epochs))[::3])
    x = np.arange(initial_epochs-1, total_epochs-1, 0.1)
    ax.fill_between(x, *ax.get_ylim(), alpha=0.2, color='green', label='fine-tuning')
    ax.legend();
    
plt.tight_layout()

**Evaluate on Test Dataset**

Evaluate your final model on test dataset and store the results in `test_eval`.

You should be able to achieve:
- Age MAE < 14
- Ethnicity Accuracy > 0.3
- Gender Accuracy > 0.6

In [34]:
### TEST FUNCTION: test_test_eval
# DO NOT REMOVE THE LINE ABOVE
# your code here
test_eval = 
print(f"Test Age MAE: {test_eval[4]:.2f}")
print(f"Test Ethnicity ACC: {test_eval[5]:.2f}")
print(f"Test Gender ACC: {test_eval[6]:.2f}")

**Predict on a Test Batch**

Take one batch from the test dataset and store it as `test_batch`.

Next, use your model to predict on this batch and store the predictions in `y_pred`.

In [37]:
### TEST FUNCTION: test_y_pred
# DO NOT REMOVE THE LINE 
test_batch = ...
y_pred = ...

### There is no more code for you to enter after this point. If your notebook runs to the end you pass the final test!
**Processing Prediction Vectors**

Here we create vectors `age_pred`, `eth_pred`, and `gen_pred` each containing the predicted values of the 3 target variables in the test batch.

These vectors are all be flat (1D) and they should all contain *integers*.

In [39]:
age_pred = np.round(y_pred[0].flatten()).astype(int)
eth_pred = y_pred[1].argmax(axis=1)
gen_pred = (y_pred[2].flatten() >= 0.5).astype(int)

Use this code to extract the images and target variables from the batch. Note the shapes of these objects. You may want to investigate them further.

In [41]:
# input images and targets from batch
images, labels =  next(test_batch.as_numpy_iterator())

In [42]:
print('images.shape:', images.shape)
print('len(labels):', len(labels))
print('labels[0].shape:', labels[0].shape)
print('labels[1].shape:', labels[1].shape)
print('labels[2].shape:', labels[2].shape)

**True Labels**

Here are the true vectors `age_true`, `eth_true`, and `gen_true` each containing the true values of the 3 target variables in the test batch.

These vectors are all be flat (1D) and they all contain integers

In [43]:
age_true, eth_true, gen_true = [y.flatten().astype(int) for y in labels]

Run the code below to inspect some example predictions from your validation batch

In [47]:
fig, axs = plt.subplots(4,4, figsize=(16,16))
axs = axs.flatten()
for i in range(16):
    axs[i].imshow(images[i])
    pred_str = f'{age_pred[i]} {names_ethnicity[eth_pred[i]]} {name_genders[gen_pred[i]]}'
    true_str = f'{age_true[i]} {names_ethnicity[eth_true[i]]} {name_genders[gen_true[i]]}'
    axs[i].set_title('[PRED]\n' + pred_str+'\n'+'[TRUE]\n'+ true_str)
    axs[i].set_xticks([])
    axs[i].set_yticks([])
plt.tight_layout()

The results are probably not the greatest given the time and resource constrains, but with such a time model and dataset it is still surprising! 

In [50]:
### TEST FUNCTION: test_you_did_it
# DO NOT REMOVE THE LINE 